In [7]:
import pandas as pd

data = pd.read_csv('../../Dataset/amd_stock_price.csv')
print(data.head())
print(data.info())

         Date  Price   Open   High    Low    Vol. Change %
0  04/15/2025  95.29  94.66  96.85  94.62  28.12M    0.84%
1  04/14/2025  94.50  97.11  97.37  93.45  36.00M    1.18%
2  04/11/2025  93.40  90.51  94.76  90.50  57.89M    5.30%
3  04/10/2025  88.70  91.00  92.95  85.06  65.21M   -8.41%
4  04/09/2025  96.84  79.22  97.91  78.87  97.83M   23.82%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1077 entries, 0 to 1076
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      1077 non-null   object 
 1   Price     1077 non-null   float64
 2   Open      1077 non-null   float64
 3   High      1077 non-null   float64
 4   Low       1077 non-null   float64
 5   Vol.      1077 non-null   object 
 6   Change %  1077 non-null   object 
dtypes: float64(4), object(3)
memory usage: 59.0+ KB
None


In [20]:
print(data['Change %'].head())  # Xem trước dữ liệu
print(data['Change %'].dtype)  # Kiểm tra kiểu dữ liệu

Date
2025-04-15     0.84
2025-04-14     1.18
2025-04-11     5.30
2025-04-10    -8.41
2025-04-09    23.82
Name: Change %, dtype: float64
float64


In [21]:
data['Change %'] = data['Change %'].astype(str).str.replace('%', '').astype(float)

In [22]:
data['Change %'] = data['Change %'].fillna('0').astype(str).str.replace('%', '').astype(float)

In [23]:
import pandas as pd

# Đọc dữ liệu
data = pd.read_csv('../../Dataset/amd_stock_price.csv')

# Kiểm tra dữ liệu
print(data.head())
print(data.info())

# Loại bỏ ký tự '%' và chuyển đổi sang kiểu float
data['Change %'] = data['Change %'].astype(str).str.replace('%', '').astype(float)

# Kiểm tra lại dữ liệu sau khi xử lý
print(data['Change %'].head())

         Date  Price   Open   High    Low    Vol. Change %
0  04/15/2025  95.29  94.66  96.85  94.62  28.12M    0.84%
1  04/14/2025  94.50  97.11  97.37  93.45  36.00M    1.18%
2  04/11/2025  93.40  90.51  94.76  90.50  57.89M    5.30%
3  04/10/2025  88.70  91.00  92.95  85.06  65.21M   -8.41%
4  04/09/2025  96.84  79.22  97.91  78.87  97.83M   23.82%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1077 entries, 0 to 1076
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      1077 non-null   object 
 1   Price     1077 non-null   float64
 2   Open      1077 non-null   float64
 3   High      1077 non-null   float64
 4   Low       1077 non-null   float64
 5   Vol.      1077 non-null   object 
 6   Change %  1077 non-null   object 
dtypes: float64(4), object(3)
memory usage: 59.0+ KB
None
0     0.84
1     1.18
2     5.30
3    -8.41
4    23.82
Name: Change %, dtype: float64
